In [1]:
import numpy as np
import torch
import random
import time
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

In [2]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

C:\Users\amps\anaconda3\envs\dl_study\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amps\.cache\huggingface\hub\models--skt--kogpt2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [3]:
sent = '근육이 커지기 위해서는'

input_ids = tokenizer.encode(sent, return_tensors='pt')
print(input_ids)

tensor([[33245, 10114, 12748, 11357]])


In [ ]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [5]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

# 2. Numpy로 Top 5 뽑기

위의 실습에서 확인한 바와 같이 '근육이 커지기 위해서는' 이라는 입력을 넣었을 때 GPT는 다음 단어로 '무엇보다' 라는 단어를 예측했었는데요.\
실제로는 수많은 후보의 다음 단어들이 있었지만, 그 중 가장 확률이 높은 단어. 즉, Top 1의 단어인 '무엇보다'를 예측한 것입니다.\
그렇다면 다음 단어로 또 어떤 후보들이 있었는지 Top 5의 단어들을 뽑아봅시다

In [6]:
output = model(input_ids)

# logits.shape == torch.Size([51200]). 즉, 총 단어 집합 크기만큼의 차원을 가지는 벡터.
logits = output.logits[0, -1]

top5 = torch.topk(logits, k=5)
tokens = [tokenizer.decode(token_id) for token_id in top5.indices.tolist()]
print(tokens)

['무엇보다', '우선', '반드시', '피부', '무엇보다도']


# 3. Numpy Top 5로 문장 생성하기

In [7]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent, return_tensors='pt')

while len(input_ids[0]) < 50:
    with torch.no_grad():
        output = model(input_ids)
    logits = output.logits[0, -1]
    top5 = torch.topk(logits, k=30)
    token_id = random.choice(top5.indices.tolist())
    input_ids = torch.cat([input_ids, torch.tensor([[token_id]])], dim=1)

tokenizer.decode(input_ids[0])

'근육이 커지기 위해서는 규칙적인 식생활 습도를 높여주는 것을 권하고있다.\n과체지방을 줄이기 위해서는 단백질 보충도 매우 중요시된다.\n아연은 혈전을 용해시키고 세포를 응고하는데 큰 관여한다..^ )\n우려대로 \\#미세조류를'